# Install packages, load dependencies and prepare directory

Install HugsVision:

In [ ]:
!pip install hugsvision

Import dependencies and set path:

In [ ]:
from hugsvision.nnet.ObjectDetectionTrainer import ObjectDetectionTrainer
from hugsvision.inference.ObjectDetectionInference import ObjectDetectionInference
from transformers import DetrFeatureExtractor, DetrForObjectDetection
import os
import shutil
import sys
import matplotlib.pyplot as plt
import json
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/Colab Notebooks/object_detection")
%matplotlib inline

# Training

Train the model for 10 epochs (`object_detection` folder needs to contain the Craters dataset in COCO format):

In [ ]:
trainer = ObjectDetectionTrainer(
    model_path="facebook/detr-resnet-101",
    model_name="MyDETRModel",
    output_dir="./out/",
    train_path="./craters/train/",
    dev_path="./craters/valid/",
    test_path="./craters/test/",
    max_epochs=10,
    batch_size=4
)

Copy `/config.json` to `/preprocessor_config.json` to make the model folder loadable for inference:

In [ ]:
trained_model_path = "./out/MYDETRMODEL/"+os.listdir("./out/MYDETRMODEL")[-1]
shutil.copy(trained_model_path + "/config.json", trained_model_path + "/preprocessor_config.json")

# Run Inference on Test Set

Load the model for inference:

In [ ]:
inference = ObjectDetectionInference(
    DetrFeatureExtractor.from_pretrained(trained_model_path),
    DetrForObjectDetection.from_pretrained(trained_model_path, from_tf=False)
)

Get predictions of bounding boxes and probabilities for the Test Set:

In [ ]:
predictions = {}
image_files = sorted([f for f in os.listdir('./craters/test/') if f.endswith('.jpg')], key = lambda x: int(x.split("_")[0]))
print(image_files)
for i, test_image in enumerate(image_files):
    image, probb, bboxes = inference.predict(
        './craters/test/'+test_image,
        threshold = 0.5,
        visualize=False
    )
    print(i)
    predictions[i] = {"file": test_image, "probb": probb.tolist(), "bboxes": bboxes.tolist()}

Convert predictions to COCO prediction format required for Object Detection Metrics Frameworks:

In [ ]:
annotations_flat = []
for i, (bboxes, probbs) in enumerate([(v["bboxes"], v["probb"]) for v in predictions.values()]):
    for box, prob in zip(bboxes, probbs):
        annotations_flat.append({
            "id": len(annotations_flat),
            "segmentation": [],
            "area": None,
            "iscrowd": 0,
            "ignore": 0,
            "image_id": i,
            "bbox": [int(round(x, 0)) for x in [box[0], box[1], box[2]-box[0], box[3]-box[1]]],
            "category_id": 0,
            "score": prob[1]
        })

predictions_reformatted = {
    "images": [{"id": i, "file_name": f, "width": 640, "height": 640} for i, f in enumerate([v["file"] for v in predictions.values()])],
    "annotations": annotations_flat,
    "categories": [{"id": 0, "name": "crater", "supercategory": "none"}]
}

Write Test Set predictions to file:

In [ ]:
with open('coco_dets.json', 'w') as f:
    json.dump(predictions_reformatted, f)